# Bicycle Infrastructure Dating
This notebook is for examining all of the bicycling network files available and determining the approximate build date (when facility was available for use).

For approximate build date, we just need dates for anything built between 2012-2016.

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'Cycling_Infra_Dating'

In [ ]:
studyarea_geo = gpd.read_file(config['studyarea']).to_crs('epsg:4326')#.unary_union

# Add reference osm files

In [ ]:
# years = [2014,2015,2016,2023]

# for year in years:
#     raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{year}.gpkg",layer='raw')
#     raw_osm.to_file(export_fp/'reference_layers.gpkg',layer=f"osm_{year}")
#     del raw_osm

# Create a version of OSM for editing and adding data to (only run once)

In [ ]:
# osm = gpd.read_file(export_fp/'reference_layers.gpkg',layer='osm_2023')
# overwrite = False
# if overwrite:
#     osm.to_file(export_fp/'osm_edit.gpkg')

# City of Atlanta / Atlanta Regional Commission / Garber Processing
NOTE: There are some data quality issues with the CoA and ARC datasets in that sometimes facilities are marked as there when they aren't. Refine this in the future.

## City of Atlanta 
Has year installed date and other notes. Contains sharrows. Also contains "planned" infrastructure that may not have actually been completed. Some street names have changed like "Confederate" to "United"

Links to Infra Installation Dates (move to an excel sheet):
- https://www.letspropelatl.org/infra-tracker

In [ ]:
#TODO, resolve difference between the old inventory and the old as some streets had their facilities upgraded recently (Edgewood) and (Confederate/United)
coa = gpd.read_file('D:/RAW/City_of_Atlanta/coa_bike_facilities_new.geojson',mask=studyarea_geo)
coa.to_crs(config['projected_crs_epsg'],inplace=True)

# only keep existing and ones where the year is defined
coa = coa[(coa['Status']=='Existing') & coa['YearInstalled'].notna()]
# remove uncessary columns
coa.drop(columns=['GlobalID','Shape__Length','LengthMi','CrossSectionNotes','Status'],inplace=True)

#rename the id/year column
coa.rename(columns={'OBJECTID':'id','YearInstalled':'year'},inplace=True)

#mark facilities that need to be dated
coa['need_date'] = (coa['year'] >= 2012) & (coa['year'] <= 2016)

# rename columns for consistency
coa.columns = ['coa_'+col.lower() if col != 'geometry' else 'geometry' for col in coa.columns.tolist()]

# convert facility type to OSM (use highest protection if two different types)
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
coa_conversion = {
       'Protected Bike Lane': osm_types[3], 
       'Protected Bike Lane / Bike Lane': osm_types[3],
       'Two-Way Cycle Track': osm_types[3], 
       'Uphill Bike Lane / Downhill Sharrows': osm_types[1],
       'Sharrows': osm_types[0], 
       'Bike Lane': osm_types[1], 
       'Bike Lane ': osm_types[1],
       'Uphill Buffered Bike Lane / Downhill Sharrows': osm_types[2],
       'Buffered Bike Lane': osm_types[2], 
       'Buffered Contraflow Bike Lane / Bike Lane': osm_types[1],
       'Shared-Use Path': osm_types[4], 
       'Neighborhood Greenway': osm_types[0], 
       'Bike Lane / Sharrows': osm_types[1],
       'Shared-Use Path / Bike Lane': osm_types[4], 
       'Buffered Bike Lane / Bike Lane': osm_types[2],
       'Buffered Bike Lane / Shared-Use Path': osm_types[4],
       'Shared-Use Path / Sharrows': osm_types[4],
       'Uphill Protected Bike Lane / Downhill Sharrows': osm_types[3], 
       'Shared Path': osm_types[4]
}
coa['coa_osm_type'] = coa['coa_facilitytype'].map(coa_conversion)

#export
coa.to_file(export_fp/'reference_layers.gpkg',layer='coa')

## Atlanta Regional Commission

In [ ]:
arc = gpd.read_file('D:\RAW\Atlanta_Regional_Comission\Bike_Ped_Trail_Inventory_January2024.geojson',mask=studyarea_geo)
arc.to_crs(config['projected_crs_epsg'],inplace=True)

# the only na values are OTP
#arc[arc['Year_2'].isna()].explore()

# remove post 2016
arc = arc[arc['Year_2'] < 2016]

#clean the columns
arc = arc[['OBJECTID_1','Name', 'spec','Width', 'Material', 'Year_2', 'geometry']]

#rename the id column
arc.rename(columns={'OBJECTID_1':'id','Year_2':'year'},inplace=True)

#mark facilities that need to be dated
arc['need_date'] = (arc['year'] >= 2012) & (arc['year'] <= 2016)

# rename columns for consistency
arc.columns = ['arc_'+col.lower() if col != 'geometry' else 'geometry' for col in arc.columns.tolist()]

# remove these facilities
remove = ['Paved shoulder','Park Trail','Campus Path']
arc = arc[arc['arc_spec'].isin(remove)==False]

# convert to osm name
arc_conversion = {
    'Hard surface multi-use path': 'multi use path',
    'Protected bike lane': 'cycletrack',
    'Uphill bike lanes / downhill sharrows': 'bike lane',
    'Conventional bike lane': 'bike lane',
    'Buffered bike lane': 'buffered bike lane',
    'Bike lane with parking': 'bike lane',
    'Buffered bike lane (BUS ONLY lane)': 'buffered bike lane',
    #'Paved shoulder': 'bike lane', # consider dumping these
    'Two way cycle track': 'cycletrack',
    'Bike lane': 'bike lane',
    'Buffered/bike lane': 'buffered bike lane',
    'Shared use path or greenway': 'multi use path',
    'Side path': 'multi use path',
    'Bike lane and Side path': 'bike lane',
    'Uphill bike lane/downhill sharrow': 'bike lane',
    'Side Path': 'multi use path',
    'Climbing lane': 'bike lane',
    'Sidepath': 'multi use path',
    #'Park Trail': 'multi use path',
    'Bike Lane with parking': 'bike lane',
    #'Campus Path': 'multi use path',
    'Bike Lane': 'bike lane'
    #'Park Path': 'multi use path'
}
arc['arc_osm_type'] = arc['arc_spec'].map(arc_conversion)

#export
arc.to_file(export_fp/'reference_layers.gpkg',layer='arc')

## Michael Garber (come back to later for the osm conversion)

In [ ]:
# garber = gpd.read_file('D:/RAW/Michael Garber/data_for_reid.shp',mask=studyarea_geo)
# garber.to_crs(config['projected_crs_epsg'],inplace=True)

# # remove post 2016
# garber = garber[(pd.to_datetime(garber['rbbn_dt']).apply(lambda row: row.year) <= 2016) | (garber['rbbn_dt'].isna())]

# #clean the columns
# garber = garber[['edge_id','infr_6_', 'in_6___', 'osm_nm_sm',
#        'rbbn_dt', 'geometry']]

# #rename
# garber.columns = ['id','Infra1','Infra2','Name','Ribbon Date','geometry']

# #mark facilities that need to be dated
# garber['need_date'] = (pd.to_datetime(garber['Ribbon Date']).apply(lambda row: row.year) >= 2012) | (garber['Ribbon Date'].isna())

# # rename columns for consistency
# garber.columns = ['garber_'+col.lower() if col != 'geometry' else 'geometry' for col in garber.columns.tolist()]

# #export
# garber.to_file(export_fp/'reference_layers.gpkg',layer='garber')

In [ ]:
# garber['garber_infra1'].append(garber['garber_infra2']).unique().tolist()
# garber[garber['garber_infra1']=='off_street_trail_dirt'].explore()
# cycleways_osm['facility_fwd'].append(cycleways_osm['facility_rev']).unique().tolist()

# Suggested Matches

Name check functions

In [ ]:
def overwrite_check(overwrite_edit_version,confirm):
    if (overwrite_edit_version == False) | (overwrite_edit_version == False):
        print('Overwrite or confirm set to false')
        return False
    elif (overwrite_edit_version) & (confirm == True):
        print('WARNING: OVERWRITE ENABLED')
        return True
    else:
        return False

In [ ]:
import string
# List of common suffixes and their abbreviated forms
suffixes = {
    'street': ['st'],
    'avenue': ['ave'],
    'boulevard': ['blvd'],
    'drive': ['dr'],
    'lane': ['ln'],
    'road': ['rd'],
    'court': ['ct'],
    'circle': ['cir'],
    'way': ['wy'],
    'place': ['pl']
    # Add more suffixes and their abbreviations as needed
}

# Function to remove suffixes from street names
def remove_suffix(street_name):
    
    if street_name is None:
        return None

    # Lowercase evertyhing
    street_name = street_name.lower()

    # remove periods
    street_name = street_name.translate(str.maketrans('','',string.punctuation))

    # Split the street name into words
    words = street_name.split()

    # Remove directional indicators
    directions = ['north', 'south', 'east', 'west', 'northeast', 'southeast', 'northwest', 'southwest', 'n', 'e', 's', 'w', 'ne', 'se', 'nw', 'sw','wb']
    for direction in directions:
        words = [word for word in words if word.lower() != direction]



    # Remove suffixes
    for suffix, abbreviations in suffixes.items():
        # Remove full suffix
        if words[-1].lower().endswith(suffix):
            words[-1] = words[-1][:-(len(suffix))]
        # Remove abbreviated suffix
        for abbr in abbreviations:
            if words[-1].lower().endswith(abbr):
                words[-1] = words[-1][:-(len(abbr))]

    # Reconstruct the street name with spaces
    cleaned_street_name = ' '.join(words)
    return cleaned_street_name.strip()  # Remove any leading or trailing whitespace

# # Example list of street names
# street_names = ['Berne St (WB)','Bill Kennedy','Main St', 'Elm Avenue', 'Maple Blvd', 'Oak Dr', 'Pine Ln', 'Northwest 1st St', 'ne 2nd Ave', 'Eagle Row']

# # Remove suffixes from street names
# cleaned_street_names = [remove_suffix(name) for name in street_names]

# # Print cleaned street names
# for name in cleaned_street_names:
#     print(name)

def name_check(name1,name2):
    if (name1 is None) | (name2 is None):
        return False
    name1_words = name1.split()
    name2_words = name2.split()
    #check if any part of the name is right
    check1 = [True for name1_word in name1_words if name1_word in name2_words]
    #check2 = [True if name2_word in name1_words else False for name2_word in name2_words]
    if len(check1) > 0:
        return True
    else:
        return False

In [ ]:
import string
def suggested_matches(cycleways_osm,other_source,other_name,buffer_ft,max_hausdorff_dist):
    
    cycleways_osm_buffered = cycleways_osm.copy()
    other_source = other_source.copy()

    # buffer the cycleways
    cycleways_osm_buffered.geometry = cycleways_osm_buffered.buffer(buffer_ft)
    
    # intersect with coa (returns coa linestrings)
    overlap = gpd.overlay(other_source,cycleways_osm_buffered)

    # create new fields
    overlap['accept_match'] = None
    overlap['examined'] = None
    overlap['notes'] = None # always good to have an extra field for notes
    
    #street name check if for bike lanes / sharrows / cycletracks
    overlap['name'] = overlap['name'].apply(lambda row: remove_suffix(row))
    overlap[f"{other_name}_name"] = overlap[f"{other_name}_name"].apply(lambda row: remove_suffix(row))
    overlap['name_check'] = overlap.apply(lambda row: name_check(row['name'],row[f"{other_name}_name"]),axis=1)

    ## AUTO REJECTS ##

    # reject duplicate matches (same other feature by attributes but different id)
    overlap.loc[overlap.drop(columns=[f"{other_name}_id",'geometry']).duplicated(),'accept_match'] = False

    # reject if one facility is a mup or cycletrack and the other is a bike lane/sharrow
    # cycletrack grouped with mups because they might get mis-classified as one
    #TODO this might rmeove matches where there is a cycletrack on one side of the road but not the other?
    off_street_infra = ['multi use path','cycletrack']
    street_infra = ['bike lane','buffered bike lane','sharrow']
    osm_street_infra = overlap[["facility_fwd",'facility_rev']].isin(street_infra).any(axis=1)
    other_street_infra = overlap[f"{other_name}_osm_type"].isin(street_infra)
    osm_is_mup = (overlap[["facility_fwd",'facility_rev']].isin(off_street_infra)).any(axis=1)
    other_is_mup = overlap[f"{other_name}_osm_type"].isin(off_street_infra)
    condition = (osm_street_infra & other_is_mup) | (other_street_infra & osm_is_mup)
    overlap.loc[condition,'accept_match'] = False

    # reject match if street infrastructure and name does not match
    overlap.loc[osm_street_infra & other_street_infra & (overlap['name_check']==False),'accept_match'] = False

    # reject if osm says sharrow for both directions and other is not a sharrow
    overlap.loc[(overlap[['facility_fwd','facility_rev']] == 'sharrow').all(axis=1) & (overlap[f"{other_name}_osm_type"] != 'sharrow'),'accept_match'] = False
    # and reject if other says sharrow but osm does not
    overlap.loc[(overlap[['facility_fwd','facility_rev']] != 'sharrow').all(axis=1) & (overlap[f"{other_name}_osm_type"] == 'sharrow'),'accept_match'] = False

    # mark the remaining one to many matches (don't count rejects)
    only_one_na = overlap.groupby('osmid')['accept_match'].transform(lambda x: x.isna().sum()==1)
    is_duplicated = overlap['osmid'].duplicated(keep=False)
    overlap['one_to_many'] = ~(only_one_na | (is_duplicated == False))

    ## AUTO ACCEPTS

    # accept if name check is correct and it's one to one
    overlap.loc[overlap['name_check'] & (overlap['one_to_many']==False) & overlap['accept_match'].isna(),'accept_match'] = True
    # accept if both are multi-use paths and it's a one to one match
    overlap.loc[osm_is_mup & other_is_mup & (overlap['one_to_many']==False) & overlap['accept_match'].isna(),'accept_match'] = True

    # replace with osm geometry
    overlap = pd.merge(overlap,cycleways_osm[['osmid','geometry']],on='osmid')
    
    overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
    overlap.drop(columns=['geometry_x'],inplace=True)
    overlap.rename(columns={'geometry_y':'geometry'},inplace=True)
    overlap = gpd.GeoDataFrame(overlap,geometry='geometry')

    #for non-assigned values give it to the one with the min hausdorff distance as long as it doesn't exceed the maximum amount    
    min_hausdorff = overlap[overlap['accept_match']!=0].groupby('osmid')['hausdorff_dist'].idxmin().tolist()
    no_match = overlap.groupby('osmid')['accept_match'].transform(lambda x: (x != 1).all())
    overlap.loc[overlap.index.isin(min_hausdorff) & (overlap['hausdorff_dist']<=max_hausdorff_dist) & no_match & overlap['accept_match'].isna(),'accept_match'] = True

    #auto assign anything that wasn't accepted as false
    match = overlap.groupby('osmid')['accept_match'].transform(lambda x: (x == 1).any())
    overlap.loc[match & overlap['accept_match'].isna(),'accept_match'] = False

    return overlap

# from shapely.geometry import LineString, MultiLineString
# #from shapely import hausdorff_distance

# line1 = LineString([(0, 0), (1, 1), (2, 2)])
# multiline = MultiLineString([[(0, 0), (1, 2)], [(2, 3), (3, 4)]])

# distance = multiline.hausdorff_distance(line1)
# print("Hausdorff Distance:", distance)

### Settings

In [ ]:
buffer_ft = 100
max_hausdorff_dist_ft = 3000

In [ ]:
#network_osm = gpd.read_file(Path(config['project_directory'])/f"Network/networks.gpkg",layer='osm_links')

# import the 2023 OSM bicycle network
cycleways_osm = gpd.read_file(export_fp/'osm_cycleways.gpkg')
cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023',['osmid','highway','name','facility_fwd','facility_rev','geometry']]

# add cycleway info
#cycleways_osm = pd.merge(network_osm,cycleways_osm,on='osmid')

In [ ]:
#import processed versions
coa = gpd.read_file(export_fp/'reference_layers.gpkg',layer='coa')
arc = gpd.read_file(export_fp/'reference_layers.gpkg',layer='arc')
#garber = gpd.read_file(export_fp/'reference_layers.gpkg',layer='garber')

#perform overlap
coa_overlap = suggested_matches(cycleways_osm,coa,'coa',buffer_ft,max_hausdorff_dist_ft)
arc_overlap = suggested_matches(cycleways_osm,arc,'arc',buffer_ft,max_hausdorff_dist_ft)
#garber_overlap = suggested_matches(cycleways_osm,garber,'garber_id','garber_name',buffer_ft)

#export (only run once to prevent lost progress)
overwrite = True
if overwrite:
    coa_overlap.to_file(export_fp/'suggested_matches.gpkg',layer='coa')
    arc_overlap.to_file(export_fp/'suggested_matches.gpkg',layer='arc')
    #garber_overlap.to_file(export_fp/'suggested_matches.gpkg',layer='garber')

#update method (to prevent overwriting)
#probably don't need this but use for adding new fields instead of overwriting the exiting one

In [ ]:
print('Total Matches:')
print('coa:',coa_overlap.shape[0],'arc:',arc_overlap.shape[0])
print('Undecided:')
print('coa:',coa_overlap['accept_match'].isna().sum(),'arc:',arc_overlap['accept_match'].isna().sum())
print('Accept:')
print('coa:',(coa_overlap['accept_match']==1).sum(),'arc:',(arc_overlap['accept_match']==1).sum())
print('Reject:')
print('coa:',(coa_overlap['accept_match']==0).sum(),'arc:',(arc_overlap['accept_match']==0).sum())

## Go through each in QGIS and confirm matches before running next step

# Missing Infrastructure (ID version)

In [ ]:
#load the suggested infra
coa_overlap = gpd.read_file(export_fp/'suggested_matches.gpkg',layer='coa')
arc_overlap = gpd.read_file(export_fp/'suggested_matches.gpkg',layer='arc')
#garber_overlap = gpd.read_file(export_fp/'suggested_matches.gpkg',layer='garber')

#get ids of accepted matches
suggested_coa_ids = coa_overlap.loc[coa_overlap['accept_match'] == '1','coa_id'].unique().tolist()
suggested_arc_ids = arc_overlap.loc[arc_overlap['accept_match'] == '1','arc_id'].unique().tolist()
#suggested_garber_ids = garber_overlap.loc[garber_overlap['accept_match'] == '1','garber_id'].unique().tolist()

In [ ]:
#import raw versions
coa = gpd.read_file(export_fp/'reference_layers.gpkg',layer='coa')
arc = gpd.read_file(export_fp/'reference_layers.gpkg',layer='arc')
#garber = gpd.read_file(export_fp/'reference_layers.gpkg',layer='garber')

In [ ]:
#what's not covered
coa_inv = coa[coa['coa_id'].isin(suggested_coa_ids) == False].copy()
arc_inv = arc[arc['arc_id'].isin(suggested_arc_ids) == False].copy()
#garber_inv = garber[garber['garber_id'].isin(suggested_garber_ids) == False]
print(coa_inv.shape[0],arc_inv.shape[0])#,garber_inv.shape[0])

In [ ]:
overwrite = True
if overwrite:
    coa_inv['included'] = None
    coa_inv['suggested_osmid'] = None
    coa_inv['notes'] = None
    coa_inv.to_file(export_fp/'id_difference.gpkg',layer='coa')
    
    arc_inv['included'] = None
    arc_inv['suggested_osmid'] = None
    arc_inv['notes'] = None
    arc_inv.to_file(export_fp/'id_difference.gpkg',layer='arc')

    # garber_inv['included'] = None
    # garber_inv['suggested_osmid'] = None
    # garber_inv['notes'] = None
    # garber_inv.to_file(export_fp/'id_difference.gpkg',layer='garber')


### Further manual processing

# Add dates to OSM cycleways network

In [ ]:
cycleways_osm = gpd.read_file(export_fp/'osm_cycleways.gpkg')
cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023',['osmid','highway','name','facility_fwd','facility_rev','geometry']]

In [ ]:
coa_overlap.columns

In [ ]:
# merge_cols = ['coa_id', 'coa_facilitytype', 'coa_builtby', 'coa_name',
#        'coa_onroad', 'coa_bothside', 'coa_year', 'coa_need_date',
#        'coa_osm_type', 'osmid']
accepted_coa = coa_overlap.loc[coa_overlap['accept_match']=='1',['osmid','coa_year']]
cycleways_osm_coa = pd.merge(cycleways_osm,accepted_coa,on='osmid',how='left')

In [ ]:
# merge_cols = ['arc_id', 'arc_name', 'arc_spec', 'arc_width', 'arc_material',
#        'arc_year', 'arc_need_date', 'arc_osm_type', 'osmid']
accepted_arc = arc_overlap.loc[arc_overlap['accept_match']=='1',['osmid','arc_year']]
cycleways_osm_coa_arc = pd.merge(cycleways_osm_coa,accepted_arc,on='osmid',how='left')

In [ ]:
cycleways_osm_coa_arc[cycleways_osm_coa_arc[['arc_year','coa_year']].notnull().any(axis=1)].explore()

In [ ]:
import numpy as np

#if both coa and arc date, take the coa date, otherwise arc
def date_fix(row):
    if row['coa_year'] != row['arc_year']:
        return row['coa_year']
    else:
        return row['arc_year']
cycleways_osm_coa_arc['year'] = cycleways_osm_coa_arc.apply(lambda row: date_fix(row),axis=1)
cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['year'].isna(),'year'] = np.nan

In [ ]:
# find the osm earliest appearance, if it's post 2020 then it definitely wasn't in the study area

In [ ]:
#TODO Set certain facilities to year=0 if it's not a bike lane or PATH trail (i.e., facilities not present in the newer data sources)
cycleways_osm_coa_arc.columns

In [ ]:
# cycleways_osm_coa_arc['min_date'] = cycleways_osm_coa_arc[['arc_year','coa_year']].min(axis=1)
# cycleways_osm_coa_arc['max_date'] = cycleways_osm_coa_arc[['arc_year','coa_year']].max(axis=1)

# #set null value
# cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['min_date'].isna(),'min_date'] = 0
# cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['max_date'].isna(),'max_date'] = 0

# Export final

In [ ]:
cycleways_osm_coa_arc.to_file(export_fp/'osm_cycleways_w_dates.gpkg',layer='test_dates')

In [ ]:
# <!-- # Missing Infrastructure (Geometry Version)
# buffer_ft = 100
# overwrite_diff = False
# confirm_diff = False
# # import the 2023 OSM bicycle network
# cycleways_osm = gpd.read_file(export_fp/'osm_cycleways.gpkg')
# cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023']

# # get unary union of all features after buffering
# cycleways_osm_all = cycleways_osm.buffer(buffer_ft).unary_union
# arc_diff = arc[arc.geometry.intersects(cycleways_osm_all) == False]
# coa_diff = coa[coa.geometry.intersects(cycleways_osm_all) == False]
# garber_diff = garber[garber.geometry.intersects(cycleways_osm_all) == False]
# overwrite = False
# if overwrite:
#     coa_diff['valid_difference'] = None
#     coa_diff['notes'] = None
#     coa_diff.to_file(export_fp/'differences.gpkg',layer='coa')
    
#     arc_diff['valid_difference'] = None
#     arc_diff['notes'] = None
#     arc_diff.to_file(export_fp/'differences.gpkg',layer='arc')

#     garber_diff['valid_difference'] = None
#     garber_diff['notes'] = None
#     garber_diff.to_file(export_fp/'differences.gpkg',layer='garber')

# raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{config['geofabrik_year']}.gpkg",layer='raw')
# raw_osm.to_crs(config['projected_crs_epsg'],inplace=True)
# final_confirm = False
# if (overwrite_check(overwrite_diff,confirm_diff) == True) & (final_confirm == True):
#     raw_osm['arc_feature_id'] = None
#     raw_osm['coa_feature_id'] = None
#     raw_osm['garber_feature_id'] = None
#     raw_osm.to_file(export_fp/'differences.gpkg',layer='osm_edit')
# # raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{config['geofabrik_year']}.gpkg",layer='raw')
# # raw_osm.to_crs(config['projected_crs_epsg'],inplace=True)
# # final_confirm = True
# # if (overwrite_check(overwrite_diff,confirm_diff) == True) & (final_confirm == True):
# #     raw_osm['arc_feature_id'] = None
# #     raw_osm['coa_feature_id'] = None
# #     raw_osm['garber_feature_id'] = None
# #     raw_osm.to_file(export_fp/'differences.gpkg',layer='osm_edit')
# # #based on the 2023-01-01 Geofabrik Georgia Extract
# # #osm = gpd.read_file(Path(config['project_directory'])/'Network/networks.gpkg',layer='osm_links')
# # osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_2023.gpkg",layer='raw',ignore_geometry=True)
# # #osm = pd.merge(osm,raw_osm,on='osmid',how='left')

# # #create new fields for install dates
# # osm['install_year'] = None
# # osm['install_month'] = None
# # osm['install_day'] = None

# # #create new fields for updated fwd and rev infra types
# # osm['facility_fwd'] = None
# # osm['facility_rev'] = None

# # #create field for notes
# # osm['notes'] = None

# # #create field for edit date
# # osm['last_edited'] = None -->